# **Week 1**

#### **Objective: Evaluate the Data Set using Polynomial and Interaction Terms**

Each dataset has already been explored through univariate, bivariate, and multivariate analysis, including assissments of variable interactions and correlations with the target outcome. 

in this section, we extend that analysis by explicitly evaluating polynomial transformations and interaction terms (both categorical and numeric). The goal is to capture potential nonlinear relationships and combined effects between variables, and to assess how these features contribute to prediction the target. 

___

#### **Package Imports**

In [10]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import math
import matplotlib.pyplot as plt
import seaborn as sns

#### **Import Datasets**

In [11]:
BDB_All_Plays = pd.read_csv("../../AFL_Final_Project/BDB_All_Plays.csv") # Big Data Bowl Dataset
FNF_All_Plays = pd.read_csv("../../AFL_Final_Project/FNF_All_Plays.csv") # First and Future Dataset
PDA_All_Plays = pd.read_csv('../../AFL_Final_Project/PDA_All_Plays.csv') # Punt Data Analytics

#### **Important Functions**

In [33]:
# =======================================================
# Taken from Module 3 Milestone 1
# # Link: https://github.com/LeeMcFarling/Module_3_Milestone_1/blob/main/Milestone_01.ipynb
# 
# Split df into Numeric and Categorical Datasets, so that 
# visualizations can be catered accordingly. 
# =======================================================

# Numeric Columns
def numerify(df):
    numeric_cols = df.select_dtypes(include='number').columns
    filtered_cols = [col for col in numeric_cols if 'id' not in col.lower()] # Filter OUT 'id' columns
    df_numeric = df[filtered_cols]
    return df_numeric


# Categorical columns
def categorify(df):
    df_categorical_cols = df.select_dtypes(exclude=['number']).columns
    cat_id_cols = [col for col in df.columns if 'id' in col.lower()] #.lower() to avoid any capitalization issues
    combined_cat_cols = list(df_categorical_cols) + cat_id_cols
    df_categorical = df[combined_cat_cols]
    return df_categorical

In [30]:
# ========================================================================================
# Taken from Module 3 - Final Project Milestone 1
# Link: https://github.com/LeeMcFarling/Module_3_Milestone_1/blob/main/Milestone_01.ipynb
# 
# Prupose:
# This is meant to consolidate the 'show_null_counts_features' function from before with 
# another with 'value' and 'unique' counts later on in this analysis. 
# ========================================================================================

def profile_dataset(df):
    # Identify feature types
    feature_types = df.dtypes.apply(lambda x: 'Numeric' if np.issubdtype(x, np.number) else 'Categorical')

    # Build a summary DataFrame
    summary = pd.DataFrame({
        'Feature': df.columns,
        'Type': feature_types.values,
        'Null Values': df.isnull().sum().values,
        'Null %': (df.isnull().mean() * 100).round(2).values,
        'Count (Non-Null)': df.count().values,
        'Unique Values': df.nunique().values
    })

    # Sort Values in Summary by % of null values
    summary = summary.sort_values(by='Null %', ascending=False).reset_index(drop=True)

    # Add dataset shape info above the table
    print(f"This dataset contain {df.shape[0]} rows")
    print(f"This dataset contain {df.shape[1]} columns")

    # Display the summary
    return summary

In [40]:
# ========================================================================================
# Taken from Module 3 - Final Project Milestone 1
# Link: https://github.com/LeeMcFarling/Module_3_Milestone_1/blob/main/Milestone_01.ipynb
# 
# Function purpose is to intake a variety of related columns (Foul 1, Foul 2, etc.) and create
# an indicator flag from it. i.e. 'Did penalty occur? (Y/N)'
# ========================================================================================


def create_indicator_from_columns(df, columns, new_column_name):
    # Initialize a boolean series with False for all rows.
    indicator = pd.Series(False, index=df.index)
    
    for col in columns:
        try:
            if col in df.columns:
                indicator = indicator | df[col].notnull()
            else:
                print(f"Warning: Column '{col}' not found. Skipping.")
        except Exception as e:
            print(f"Error processing column '{col}': {e}")
    
    # Assign the indicator as an integer column to the DataFrame
    df[new_column_name] = indicator.astype(int)
    return df

#### **DataFrame Functions**

In [ ]:
pd.set_option('display.max_columns', None) 
pd.set_option('display.max_colwidth', None)


____

# **Big Data Bowl PreAnalysis**

In [35]:
BDB_All_Plays.head(1)

,gameId,playId,playDescription,quarter,down,yardsToGo,possessionTeam,defensiveTeam,yardlineSide,yardlineNumber,gameClock,preSnapHomeScore,preSnapVisitorScore,passResult,penaltyYards,prePenaltyPlayResult,playResult,foulName1,foulNFLId1,foulName2,foulNFLId2,foulName3,foulNFLId3,absoluteYardlineNumber,offenseFormation,personnelO,defendersInBox,personnelD,dropBackType,pff_playAction,pff_passCoverage,pff_passCoverageType,Inj_Occured
0,2021090900,97,(13:33) (Shotgun) T.Brady pass incomplete deep right to C.Godwin.,1,3,2,TB,DAL,TB,33,13:33,0,0,I,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,43.0,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"4 DL, 2 LB, 5 DB",TRADITIONAL,0,Cover-1,Man,0


Let's break the dataset down into numeric and categorical data

In [34]:
BDB_All_Plays_Numeric = numerify(BDB_All_Plays)
BDB_All_Plays_Categorical = categorify(BDB_All_Plays)

Here is a profile of the numeric data

In [36]:
profile_dataset(BDB_All_Plays_Numeric)

This dataset contain 8557 rows
This dataset contain 13 columns


,Feature,Type,Null Values,Null %,Count (Non-Null),Unique Values
0,penaltyYards,Numeric,7801,91.17,756,60
1,defendersInBox,Numeric,7,0.08,8550,11
2,absoluteYardlineNumber,Numeric,1,0.01,8556,99
3,quarter,Numeric,0,0.00,8557,5
4,down,Numeric,0,0.00,8557,5
5,yardsToGo,Numeric,0,0.00,8557,32
6,yardlineNumber,Numeric,0,0.00,8557,50
7,preSnapHomeScore,Numeric,0,0.00,8557,42
8,preSnapVisitorScore,Numeric,0,0.00,8557,38
9,prePenaltyPlayResult,Numeric,0,0.00,8557,98


From looking at the data, it looks like we need to impute a 0 in the penaltyYards column if there was no injury, and either drop the rows. or inpute an unknown into the absoluteYardlineNumber and defendersInBox. 

In [37]:
profile_dataset(BDB_All_Plays_Categorical)

This dataset contain 8557 rows
This dataset contain 21 columns


,Feature,Type,Null Values,Null %,Count (Non-Null),Unique Values
0,foulNFLId3,Numeric,8556,99.99,1,1
1,foulName3,Categorical,8556,99.99,1,1
2,foulNFLId2,Numeric,8527,99.65,30,30
3,foulName2,Categorical,8527,99.65,30,15
4,foulNFLId1,Numeric,7821,91.40,736,486
5,foulName1,Categorical,7821,91.40,736,29
6,dropBackType,Categorical,528,6.17,8029,8
7,yardlineSide,Categorical,125,1.46,8432,32
8,yardlineSide,Categorical,125,1.46,8432,32
9,offenseFormation,Categorical,7,0.08,8550,7


There are a lot of null fields in the Foul / Penalty Fields. Let's roll that up into an indicator field to make things more simple to follow: 


- First, let's roll up Foul ID fields into the flag we need. 
- Second, let's consolidate Foul2 and Foul3

First, we'll use the fould ID fields to make a flag called 'foul_on_play'. 

In [43]:
columns = ['foulNFLId1', 'foulNFLId2', 'foulNFLId3']
new_column_name = 'foul_on_play'
BDB_All_Plays_Clean = create_indicator_from_columns(BDB_All_Plays, columns, new_column_name)

BDB_All_Plays_Clean = BDB_All_Plays_Clean.drop(columns=columns, index=1)

Next, because the prevalence of injuries is so small compared to the total number of plays overall, and because the number of plays in which both fouls and Injuries occured is smaller still, we should do some quick analysis to see if granular foul name type information is worth keeping, or if a simple foul flag will suffice. 

The reasoning is that if a certain foul only occured twice in the overall total sample, we cannot reliably identify patterns on such a small number of events. Keeping this level of dtail risks the temptation to bootstrap an exceedingly small sample, which could amplify incomplete or misleading interactions. within the dataset as a whole. 

In [ ]:
print(f' Number of Plays in which Injuries & Fouls Occured: {len(BDB_All_Plays_Clean[(BDB_All_Plays_Clean['Inj_Occured'] == 1) & (BDB_All_Plays_Clean['foul_on_play'] == 1)])}')
print(f' Number of Plays in which Injuries Occured: {len(BDB_All_Plays_Clean[(BDB_All_Plays_Clean['Inj_Occured'] == 1)])}')
print(f' Number of Total Plays: {len(BDB_All_Plays)}')

 Number of Plays in which Injuries Occured: 209
 Number of Plays in which Injuries & Fouls Occured: 25
 Number of Total Plays: 8557


As suspected, the number of plays in which both fouls and injuries occured is 25 out of 8557, so ~ about a 0.0029 rate of incidence. 

Further investigating the data, we can see that there are 29 foul types in FoulName1 and 15 in FoulName2 and so on. 


Example: 

In [58]:
for col in ['foulName1', 'foulName2', 'foulName3']:
    print(f'{col}: {BDB_All_Plays_Clean[col].dropna().unique()}')
    print()

foulName1: ['Illegal Use of Hands' 'Taunting' 'Defensive Pass Interference'
 'Defensive Holding' 'Offensive Holding' 'Illegal Block Above the Waist'
 'Intentional Grounding' 'Offensive Pass Interference'
 'Unsportsmanlike Conduct' 'Defensive Offside' 'Illegal Formation'
 'Roughing the Passer' 'Unnecessary Roughness' 'Illegal Touch Pass'
 'Face Mask (15 Yards)' 'Ineligible Downfield Pass' 'Illegal Contact'
 'Disqualification' 'Illegal Blindside Block'
 'Lowering the Head to Initiate Contact' 'Chop Block' 'Low Block'
 'Illegal Shift' 'Tripping' 'Illegal Forward Pass' 'Illegal Substitution'
 'Illegal Motion' 'Horse Collar Tackle' 'Clipping']

foulName2: ['Unnecessary Roughness' 'Face Mask (15 Yards)' 'Tripping'
 'Defensive Offside' 'Roughing the Passer' 'Defensive Pass Interference'
 'Offensive Holding' 'Unsportsmanlike Conduct' 'Defensive Holding'
 'Illegal Use of Hands' 'Taunting' 'Disqualification'
 'Intentional Grounding' 'Offensive Pass Interference' 'Illegal Contact']

foulName3: ['

^ As such, the data is too granular on such a small scale to combine and then create dummy variables for > 29 different categories. Further investigations into foul types on injury rates are worth investigating but the sample size should be more than 8 weeks in one NFL season. 

For now, a foul_on_play flag will be deemed sufficient, and extra granular information will be removed to avoid any risks in confounding the model. 

In [71]:
drop_cols = ['foulName1', 'foulName2', 'foulName3']
BDB_All_Plays_Clean.drop(columns=drop_cols, axis=1, inplace=True)

In [72]:
profile_dataset(BDB_All_Plays_Clean)

This dataset contain 8556 rows
This dataset contain 28 columns


,Feature,Type,Null Values,Null %,Count (Non-Null),Unique Values
0,penaltyYards,Numeric,7800,91.16,756,60
1,dropBackType,Categorical,528,6.17,8028,8
2,yardlineSide,Categorical,125,1.46,8431,32
3,defendersInBox,Numeric,7,0.08,8549,11
4,offenseFormation,Categorical,7,0.08,8549,7
5,personnelD,Categorical,1,0.01,8555,29
6,personnelO,Categorical,1,0.01,8555,30
7,absoluteYardlineNumber,Numeric,1,0.01,8555,99
8,prePenaltyPlayResult,Numeric,0,0.00,8556,98
9,Inj_Occured,Numeric,0,0.00,8556,2


Now let's check up on the features that still have null fields. 

In [74]:
for col in ['penaltyYards', 'dropBackType', 'yardlineSide', 'defendersInBox', 'offenseFormation']:
    print(f'{col}: {BDB_All_Plays_Clean[col].unique()}')
    print()

penaltyYards: [ nan   0.  14.   5.  26. -10.  16.  10.  -5. -14.   3.  19. -12.   8.
  15.  13.  11. -15.  17.  35.   2.   9.  -2.   6.  -4.  32.  21. -11.
  27.  36.   4.  12. -18.  25.  48.  -7.  28.  -3.  -6.  24.   7.   1.
  18.  43.  22.  23.  -9.  31.  20.  45.  47.  41.  33.  -8.  50.  39.
  46.  40.  38.  42. -13.]

dropBackType: ['TRADITIONAL' 'SCRAMBLE_ROLLOUT_RIGHT' 'DESIGNED_ROLLOUT_RIGHT' nan
 'SCRAMBLE' 'DESIGNED_ROLLOUT_LEFT' 'UNKNOWN' 'DESIGNED_RUN'
 'SCRAMBLE_ROLLOUT_LEFT']

yardlineSide: ['TB' 'DAL' nan 'ATL' 'PHI' 'PIT' 'BUF' 'NYJ' 'CAR' 'MIN' 'CIN' 'DET' 'SF'
 'HOU' 'JAX' 'IND' 'SEA' 'TEN' 'ARI' 'LAC' 'WAS' 'CLE' 'KC' 'MIA' 'NE'
 'NO' 'GB' 'NYG' 'DEN' 'CHI' 'LA' 'LV' 'BAL']

defendersInBox: [ 6.  7.  5.  4.  8.  3.  9. 10. nan 11.  2.  1.]

offenseFormation: ['SHOTGUN' 'SINGLEBACK' 'EMPTY' 'I_FORM' 'JUMBO' 'PISTOL' nan 'WILDCAT']



Of the five features with missing values, three are easily imputed with 0 or unknown values -- the reasoning is as follows: 

- Penalty Yards: If no penalty, there are 0 penalty yards,
- drop back type:  An 'Unknown' type already exists. If N/A, then it's assumed unknown 
- YardLineSide: This is a string type categorical variable. It is easily imputed as NA with little change to the overall nature of the field. 

- OffensiveFormation: Again, as this is a string type categorical field, N/A values are easily imputed as UNKNONWN here. 

In [ ]:
BDB_All_Plays_Clean = BDB_All_Plays_Clean.fillna({
    'penaltyYards': 0,
    'dropBackType': 'UNKNOWN',
    'yardlineSide': 'UNK',
    'offensiveFormation': 'UNKNOWN'
})

Because defenders in the box is not categorical, and is indeed numeric, adding an unknown category here would change the nature of the feature itself, additionally, we cannot impute '0' in this case because '0' is distinct from 'unknown' and doing so could confound the variable. 

To further investigate, let's query based on this field specifically

In [ ]:
BDB_All_Plays_Clean[BDB_All_Plays_Clean['defendersInBox'].isna()].index()

,gameId,playId,playDescription,quarter,down,yardsToGo,possessionTeam,defensiveTeam,yardlineSide,yardlineNumber,gameClock,preSnapHomeScore,preSnapVisitorScore,passResult,penaltyYards,prePenaltyPlayResult,playResult,absoluteYardlineNumber,offenseFormation,personnelO,defendersInBox,personnelD,dropBackType,pff_playAction,pff_passCoverage,pff_passCoverageType,Inj_Occured,foul_on_play
916,2021091211,1517,"(:20) (No Huddle, Shotgun) Aa.Rodgers pass incomplete deep right to A.Lazard. PENALTY on NO-C.Gardner-Johnson, Illegal Use of Hands, 5 yards, enforced at GB 43 - No Play.",2,3,6,GB,NO,GB,43,00:20,17,0,I,5.0,0,5,53.0,NaN,"1 RB, 1 TE, 3 WR",NaN,"3 DL, 2 LB, 6 DB",UNKNOWN,0,2-Man,Man,0,1
1570,2021091904,3676,(5:25) T.Bridgewater pass short right to A.Okwuegbunam to JAX 25 for 14 yards (A.Cisco).,4,1,10,DEN,JAX,JAX,39,05:25,13,23,C,0.0,14,14,NaN,NaN,NaN,NaN,NaN,UNKNOWN,1,Cover-1,Man,0,0
1654,2021091906,639,"(2:42) M.Jones pass incomplete deep middle to K.Bourne (B.Echols). PENALTY on NYJ-T.Ward, Defensive Offside, 4 yards, enforced at NYJ 39 - No Play.",1,2,5,NE,NYJ,NYJ,39,02:42,0,3,I,4.0,0,4,49.0,NaN,"2 RB, 1 TE, 2 WR",NaN,"3 DL, 4 LB, 4 DB",UNKNOWN,1,Cover-1,Man,0,1
4887,2021101004,3207,"(11:03) (Shotgun) T.Lawrence pass deep left intended for J.Agnew INTERCEPTED by J.Jenkins at TEN -9. Touchback. PENALTY on TEN-C.Farley, Defensive Holding, 4 yards, enforced at TEN 9 - No Play.",4,3,7,JAX,TEN,TEN,9,11:03,19,31,IN,4.0,0,0,19.0,NaN,"1 RB, 1 TE, 3 WR",NaN,"2 DL, 3 LB, 6 DB",UNKNOWN,0,Cover-1,Man,0,1
6874,2021102402,2491,"(5:45) (No Huddle, Shotgun) T.Tagovailoa scrambles right end to ATL 3 for 3 yards (D.Harmon). Penalty on MIA-J.Davis, Offensive Holding, offsetting, enforced at ATL 6 - No Play. Penalty on ATL-F.Oluokun, Defensive Holding, offsetting.",3,2,6,MIA,ATL,ATL,6,05:45,7,20,R,0.0,3,0,16.0,NaN,"1 RB, 2 TE, 2 WR",NaN,"2 DL, 4 LB, 5 DB",UNKNOWN,0,Bracket,Other,0,1
6899,2021102402,3689,"(5:37) (Shotgun) T.Tagovailoa pass left to J.Waddle pushed ob at ATL 36 for 8 yards (F.Moreau). PENALTY on MIA, Illegal Formation, 5 yards, enforced at ATL 44 - No Play.",4,1,14,MIA,ATL,ATL,44,05:37,21,27,C,-5.0,8,-5,66.0,NaN,"1 RB, 2 TE, 2 WR",NaN,"2 DL, 4 LB, 5 DB",UNKNOWN,1,Cover-3,Zone,0,0
7912,2021103105,1905,"(:11) (Shotgun) D.Mills pass short right intended for B.Cooks INTERCEPTED by D.Deayon [A.Donald] at LA 34. D.Deayon ran ob at LA 34 for no gain. PENALTY on LA-E.Jones, Illegal Use of Hands, 5 yards, enforced at LA 47 - No Play.",2,1,10,HOU,LA,LA,47,00:11,0,24,IN,5.0,0,0,57.0,NaN,"1 RB, 1 TE, 3 WR",NaN,"2 DL, 3 LB, 6 DB",UNKNOWN,0,Quarters,Zone,0,1


Judging from the INJ_OCCURED field, there were no injuries that happened in this (extremely small) sample. In this case, it's safe just to drop the records and it will not have a significant impact on our target. 

In [80]:
BDB_dropable_records = BDB_All_Plays_Clean[BDB_All_Plays_Clean['defendersInBox'].isna()].index
BDB_All_Plays_Clean = BDB_All_Plays_Clean.drop(BDB_dropable_records)

And let's re-profile the dataset to see how  we're doing: 

In [81]:
profile_dataset(BDB_All_Plays_Clean)

This dataset contain 8549 rows
This dataset contain 28 columns


,Feature,Type,Null Values,Null %,Count (Non-Null),Unique Values
0,gameId,Numeric,0,0.0,8549,122
1,playId,Numeric,0,0.0,8549,3761
2,Inj_Occured,Numeric,0,0.0,8549,2
3,pff_passCoverageType,Categorical,0,0.0,8549,3
4,pff_passCoverage,Categorical,0,0.0,8549,12
5,pff_playAction,Numeric,0,0.0,8549,2
6,dropBackType,Categorical,0,0.0,8549,8
7,personnelD,Categorical,0,0.0,8549,29
8,defendersInBox,Numeric,0,0.0,8549,11
9,personnelO,Categorical,0,0.0,8549,30


Great, we're almost ready to investigate the polynomial terms, all we have to do left is drop the ID type fields, make dummy variables for the categorical features, and then do some last minute checks to make sure we don't have perfect multicoliniarity. 

Regarding the ID fields:

In [89]:
ID_Fields = ['gameId', 'playId']
BDB_All_Plays_Clean.drop(columns=ID_Fields, inplace=True)

And now let's handly the dummy variables: 
